In [9]:
import pandas as pd

# TODO :

* market basket analysis: pour determiner les corelations entre les evenements
* Variable Importance with ML Decision Tree: separer les donées en n classe pour variable interreser. n étant les velurs unique dans la classe et evaluer l'importance des valeurs

In [10]:
def add_cols(df: pd.DataFrame, cols: list, init_val):
    for col in cols:
        df[str(col)] = init_val
        
    return df



def prefix_with_col(df: pd.DataFrame) -> pd.DataFrame:
    result = add_cols(pd.DataFrame(), df.columns, '')
    result['Num_Acc'] = df['Num_Acc']
    for col in df.columns:
        if str(col) == 'Num_Acc': continue
        result[str(col)] = df[col].map(lambda x: f'{col}__{convert_to_int(x)}')
            
    return result


# 17h, 3, 1, 18-25 -> acc grv : 70%
# 18h, 3, 0, 18-25 -> acc mg : 80%



In [12]:
data_folder = '../data/accidents-in-france-from-2005-to-2016/'

caracteristics = pd.read_csv(data_folder + 'caracteristics.csv', encoding='latin-1', low_memory=False)

In [13]:
caracteristics = caracteristics.drop(columns=['adr', 'an', 'lat',  'long', 'gps'])

In [14]:
caracteristics.head(3)

,Num_Acc,mois,jour,hrmn,lum,agg,int,atm,col,com,dep
0,201600000001,2,1,1445,1,2,1,8.0,3.0,5.0,590
1,201600000002,3,16,1800,1,2,6,1.0,6.0,5.0,590
2,201600000003,7,13,1900,1,1,1,1.0,6.0,11.0,590


In [15]:
users = pd.read_csv(data_folder + 'users.csv', encoding='latin-1', low_memory=False)

In [16]:
users = users.drop(columns=['num_veh'])

In [17]:
users.head(3)

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais
0,201600000001,1.0,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0
1,201600000001,1.0,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0
2,201600000002,1.0,1,3,1,5.0,11.0,0.0,0.0,0.0,1960.0


In [18]:
vehicles = pd.read_csv(data_folder + 'vehicles.csv', encoding='latin-1', low_memory=False)

In [19]:
vehicles = vehicles.drop(columns=['num_veh'])

In [20]:
vehicles.head(3)

,Num_Acc,senc,catv,occutc,obs,obsm,choc,manv
0,201600000001,0.0,7,0,0.0,0.0,1.0,1.0
1,201600000001,0.0,2,0,0.0,0.0,7.0,15.0
2,201600000002,0.0,7,0,6.0,0.0,1.0,1.0


pour analyser plus finement les données et pour augmenter les nombre les patterns on peut faire certaines pretraitemnt. notament transformer les annnée de naissance en tranche d'age. par exemple tranche d'age entre 18-25, 26-35, 36-50, 51-*.

In [21]:
dataset = caracteristics.merge(users, on='Num_Acc')
dataset = dataset.merge(vehicles, on='Num_Acc')

In [22]:
dataset.head(3)

,Num_Acc,mois,jour,hrmn,lum,agg,int,atm,col,com,...,actp,etatp,an_nais,senc,catv,occutc,obs,obsm,choc,manv
0,201600000001,2,1,1445,1,2,1,8.0,3.0,5.0,...,0.0,0.0,1983.0,0.0,7,0,0.0,0.0,1.0,1.0
1,201600000001,2,1,1445,1,2,1,8.0,3.0,5.0,...,0.0,0.0,1983.0,0.0,2,0,0.0,0.0,7.0,15.0
2,201600000001,2,1,1445,1,2,1,8.0,3.0,5.0,...,0.0,0.0,2001.0,0.0,7,0,0.0,0.0,1.0,1.0


In [23]:
caracteristics = None
users = None
vehicles = None

In [24]:
def convert_to_int(val):
    try:
        return int(val)
    except ValueError:
        return -1
    
def key_gen(col_name, val):
    return f'{col_name}__{val}'

def get_unique_id(unique_ids: dict, col_name: str, val):
    k = key_gen(col_name, convert_to_int(val))
    
    if k not in unique_ids.keys():
            unique_ids[k] = len(unique_ids.keys())
            
    return unique_ids[k]
    

def transform_to_transaction(df: pd.DataFrame):
    unique_ids = {}
    
    result = add_cols(pd.DataFrame(), df.columns, -1)
    result['Num_Acc'] = df['Num_Acc']
    
    for col in df.columns:
        if str(col) == 'Num_Acc': continue
            
        result[str(col)] = df[col].map(lambda x: get_unique_id(unique_ids, str(col), x))
            
    return result, unique_ids

In [26]:
dataset_sub = dataset.iloc[:50000]

In [27]:
dataset_tr, unique_ids = transform_to_transaction(dataset_sub)

In [28]:
len(unique_ids.keys())

1645

In [29]:
dataset_tr.head(3)

,Num_Acc,mois,jour,hrmn,lum,agg,int,atm,col,com,...,actp,etatp,an_nais,senc,catv,occutc,obs,obsm,choc,manv
0,201600000001,0,12,43,494,499,501,510,519,526,...,1413,1422,1427,1528,1532,1556,1582,1600,1608,1619
1,201600000001,0,12,43,494,499,501,510,519,526,...,1413,1422,1427,1528,1533,1556,1582,1600,1609,1620
2,201600000001,0,12,43,494,499,501,510,519,526,...,1413,1422,1428,1528,1532,1556,1582,1600,1608,1619


In [30]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()

In [31]:
dataset_values = dataset_tr.values

In [32]:
dataset_values = dataset_values[:, 1:] # remove Num_Acc

In [33]:
sub_sample_dataset = dataset_values[:50000]

In [34]:
sub_sample_dataset[:2]

array([[   0,   12,   43,  494,  499,  501,  510,  519,  526, 1263, 1356,
        1366, 1370, 1374, 1376, 1384, 1403, 1413, 1422, 1427, 1528, 1532,
        1556, 1582, 1600, 1608, 1619],
       [   0,   12,   43,  494,  499,  501,  510,  519,  526, 1263, 1356,
        1366, 1370, 1374, 1376, 1384, 1403, 1413, 1422, 1427, 1528, 1533,
        1556, 1582, 1600, 1609, 1620]])

In [35]:
te_ary = te.fit(sub_sample_dataset).transform(sub_sample_dataset)

In [89]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [104]:
import json

def filter_binary_dataset(binary_df: pd.DataFrame, unique_ids:dict):
    
    f = open('frequent_info.json', 'r')
    json_data = json.load(f)
    
    to_keep = []
    
    for col_name in json_data:
        col = json_data[col_name]
        
        print(col['range'])
        
        for i in range(col['range'][0], col['range'][1]+1, 1):
            k = key_gen(col_name, i)
            if k in unique_ids.keys():
                 to_keep.append(unique_ids[k])
    
    print(to_keep)
    return binary_df[to_keep]

    #to_remove = set(unique_ids.values()) - set(to_keep)
    
   # for i in to_remove:
    #    binary_df.drop(columns=i, inplace=True)
       
        #range = json_data[columnName][0]["range"]

        #data = dataset[dataset[columnName] >= range[0]]
        #dataFiltered = data[dataset[columnName]<= range[1]]
        #return dataFiltered


In [105]:
unique_ids[9]

KeyError: 9

In [106]:
df = filter_binary_dataset(df, unique_ids)

[1, 12]
[1, 31]
[1, 5]
[1, 2]
[1, 9]
[1, 9]
[1, 7]
[1, 9]
[1, 4]
[1, 3]
[1, 4]
[1, 4]
[1, 9]
[1, 7]
[1, 5]
[1, 4]
[1, 4]
[1, 2]
[1, 9]
[1, 9]
[1, 8]
[0, 9]
[1, 3]
[1, 2]
[1, 13]
[7, 0, 1, 8, 5, 11, 2, 3, 6, 10, 9, 4, 12, 25, 40, 28, 21, 24, 26, 23, 29, 31, 41, 27, 14, 17, 15, 13, 22, 33, 35, 34, 38, 37, 16, 36, 19, 39, 32, 20, 30, 18, 42, 494, 495, 496, 498, 497, 500, 499, 501, 504, 503, 507, 509, 502, 508, 506, 505, 511, 515, 516, 518, 514, 517, 512, 510, 513, 522, 521, 519, 525, 524, 520, 523, 1366, 1367, 1368, 1369, 1370, 1373, 1371, 1372, 1379, 1382, 1380, 1381, 1378, 1377, 1391, 1400, 1401, 1406, 1407, 1409, 1405, 1408, 1404, 1412, 1410, 1413, 1416, 1414, 1415, 1418, 1419, 1421, 1417, 1423, 1424, 1425, 1537, 1533, 1546, 1532, 1538, 1553]


In [107]:
df

,7,0,1,8,5,11,2,3,6,10,...,1417,1423,1424,1425,1537,1533,1546,1532,1538,1553
0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
2,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
49996,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
49997,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
49998,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [ ]:
import json

def filtrage(columnName, dataset):
    f = open('frequent_info.json')
    json_data = json.load(f)
    if columnName in json_data:
        print(columnName)
        range = json_data[columnName][0]["range"]
        
        data = dataset[dataset[columnName] >= range[0]]
        dataFiltered = data[dataset[columnName]<= range[1]]
        return dataFiltered

data = dataset_tr       
for column in dataset.columns:
    dataFiltered = filtrage(column, data)
    data = dataFiltered
data



In [70]:
freq_items = apriori(df, min_support=0.5, use_colnames=True,verbose=1, low_memory=True)
freq_items.head()

NameError: name 'apriori' is not defined

In [109]:
from mlxtend.frequent_patterns import association_rules, fpmax

In [110]:
closed_itemset = fpmax (df, min_support=0.3, use_colnames=True)
closed_itemset.shape

(25, 2)

In [111]:
id_inv_map = {v: k for k, v in unique_ids.items()}

In [112]:
def decompose_key(k:str):
    return k.split('__')[0], int(k.split('__')[1])

In [113]:
res = []
for i, itemset in closed_itemset.iterrows():
    l = [itemset['support']]
    for i in itemset['itemsets']:
        l.append(decompose_key(id_inv_map[i]))
    
    res.append(l)
    
res

[[0.31974, ('agg', 2)],
 [0.3198, ('grav', 3), ('actp', 0)],
 [0.30136, ('grav', 1), ('int', 1), ('actp', 0)],
 [0.30346, ('trajet', 5), ('lum', 1)],
 [0.31228, ('trajet', 5), ('int', 1), ('actp', 0)],
 [0.31966, ('trajet', 5), ('actp', 0), ('atm', 1)],
 [0.36502, ('trajet', 5), ('actp', 0), ('catu', 1)],
 [0.32474, ('catu', 1), ('agg', 1), ('actp', 0), ('catv', 7)],
 [0.33728, ('catv', 7), ('actp', 0), ('catu', 1), ('lum', 1)],
 [0.34328, ('catv', 7), ('actp', 0), ('catu', 1), ('atm', 1)],
 [0.37232, ('actp', 0), ('catv', 7), ('int', 1), ('catu', 1)],
 [0.31968, ('agg', 1), ('actp', 0), ('catv', 7), ('atm', 1)],
 [0.34232, ('agg', 1), ('actp', 0), ('catv', 7), ('lum', 1)],
 [0.39042, ('agg', 1), ('int', 1), ('actp', 0), ('catv', 7)],
 [0.34924, ('catv', 7), ('actp', 0), ('lum', 1), ('atm', 1)],
 [0.36992, ('actp', 0), ('catv', 7), ('int', 1), ('lum', 1)],
 [0.3656, ('catv', 7), ('int', 1), ('actp', 0), ('atm', 1)],
 [0.35482, ('agg', 1), ('actp', 0), ('catu', 1), ('atm', 1)],
 [0.3006

In [115]:
decrypt(closed_itemset)

s
u


IndexError: string index out of range

In [114]:
def decrypt(transactionsList):
    cmp = 0
    f = open("frequent_sets.txt", "w")
    
    for itemsetList in transactionsList:
        f.write("-----------------ITEM SET----------------------\n")
        for item in itemsetList:
            
            print(item)
            if cmp != 0:
                colonne = item[0]
                valeur = item[1]
                if colonne == "lum":
                    if valeur == 1:
                        f.write(colonne+" : Full day")
                    elif valeur == 2:
                        f.write(colonne+" : Twilight or dawn")
                    elif valeur == 3:
                        f.write(colonne+" : Night without public lighting")
                    elif valeur == 4:
                        f.write(colonne+" :  Night with public lighting not lit")
                    elif valeur == 5:
                        f.write(colonne+" : Night with public lighting on")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "agg":
                    if valeur == 1:
                        f.write(colonne+" : Out of agglomeration")
                    elif valeur == 2:
                        f.write(colonne+" : In built-up areas")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "int":
                    if valeur == 1:
                        f.write(colonne+" : Out of intersection")
                    elif valeur == 2:
                        f.write(colonne+" : Intersection in X")
                    elif valeur == 3:
                        f.write(colonne+" : Intersection in T")
                    elif valeur == 4:
                        f.write(colonne+" : Intersection in Y")
                    elif valeur == 5:
                        f.write(colonne+" : Intersection with more than 4 branches")
                    elif valeur == 6:
                        f.write(colonne+" : Giratory")
                    elif valeur == 7:
                        f.write(colonne+" : Place")
                    elif valeur == 8:
                        f.write(colonne+" : Level crossing")
                    elif valeur == 9:
                        f.write(colonne+" : Other intersection")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "atm":
                    if valeur == 1:
                        f.write(colonne+" : Normal")
                    elif valeur == 2:
                        f.write(colonne+" : Light rain")
                    elif valeur == 3:
                        f.write(colonne+" : Heavy rain")
                    elif valeur == 4:
                        f.write(colonne+" : Snow - hail")
                    elif valeur == 5:
                        f.write(colonne+" : Fog - smoke")
                    elif valeur == 6:
                        f.write(colonne+" : Strong wind - storm")
                    elif valeur == 7:
                        f.write(colonne+" : Dazzling weather")
                    elif valeur == 8:
                        f.write(colonne+" : Cloudy weather")
                    elif valeur == 9:
                        f.write(colonne+" : Other")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "col":
                    if valeur == 1:
                        f.write(colonne+" : Two vehicles - frontal")
                    elif valeur == 2:
                        f.write(colonne+" : Two vehicles - from the rear")
                    elif valeur == 3:
                        f.write(colonne+" : Two vehicles - by the side")
                    elif valeur == 4:
                        f.write(colonne+" : Three vehicles and more - in chain")
                    elif valeur == 5:
                        f.write(colonne+" : Three or more vehicles - multiple collisions")
                    elif valeur == 6:
                        f.write(colonne+" : Other collision")
                    elif valeur == 7:
                        f.write(colonne+" : Without collision")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "catr":
                    if valeur == 1:
                        f.write(colonne+" : Highway")
                    elif valeur == 2:
                        f.write(colonne+" : National Road")
                    elif valeur == 3:
                        f.write(colonne+" : Departmental Road")
                    elif valeur == 4:
                        f.write(colonne+" : Communal Way")
                    elif valeur == 5:
                        f.write(colonne+" : Off public network")
                    elif valeur == 6:
                        f.write(colonne+" : Parking lot open to public traffic")
                    elif valeur == 9:
                        f.write(colonne+" : other")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")

                elif colonne == "circ":
                    if valeur == 1:
                        f.write(colonne+" : One way")
                    elif valeur == 2:
                        f.write(colonne+" : Bidirectional")
                    elif valeur == 3:
                        f.write(colonne+" : Separated carriageways")
                    elif valeur == 4:
                        f.write(colonne+" : With variable assignment channels")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "vosp":
                    if valeur == 1:
                        f.write(colonne+" : Bike path")
                    elif valeur == 2:
                        f.write(colonne+" : Cycle Bank")
                    elif valeur == 3:
                        f.write(colonne+" : Reserved channel")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "Prof":
                    if valeur == 1:
                        f.write(colonne+" : Dish")
                    elif valeur == 2:
                        f.write(colonne+" : Slope")
                    elif valeur == 3:
                        f.write(colonne+" : Hilltop")
                    elif valeur == 4:
                        f.write(colonne+" : Hill bottom")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")

                elif colonne == "plan":
                    if valeur == 1:
                        f.write(colonne+" : Straight part")
                    elif valeur == 2:
                        f.write(colonne+" : Curved on the left")
                    elif valeur == 3:
                        f.write(colonne+" : Curved right")
                    elif valeur == 4:
                        f.write(colonne+" : In \"S")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")      

                elif colonne == "surf":
                    if valeur == 1:
                        f.write(colonne+" : normal")
                    elif valeur == 2:
                        f.write(colonne+" : wet")
                    elif valeur == 3:
                        f.write(colonne+" : puddles")
                    elif valeur == 4:
                        f.write(colonne+" : flooded")
                    elif valeur == 5:
                        f.write(colonne+" : snow")
                    elif valeur == 6:
                        f.write(colonne+" : mud")
                    elif valeur == 7:
                        f.write(colonne+" : icy")
                    elif valeur == 8:
                        f.write(colonne+" : fat - oil")
                    elif valeur == 9:
                        f.write(colonne+" : other")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")                
                elif colonne == "infra":
                    if valeur == 1:
                        f.write(colonne+" : Underground - tunnel")
                    elif valeur == 2:
                        f.write(colonne+" : Bridge - autopont")
                    elif valeur == 3:
                        f.write(colonne+" : Exchanger or connection brace")
                    elif valeur == 4:
                        f.write(colonne+" : Railway")
                    elif valeur == 5:
                        f.write(colonne+" : Carrefour arranged")
                    elif valeur == 6:
                        f.write(colonne+" : Pedestrian area")
                    elif valeur == 7:
                        f.write(colonne+" : Toll zone")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")   
                elif colonne == "situ":
                    if valeur == 1:
                        f.write(colonne+" : On the road")
                    elif valeur == 2:
                        f.write(colonne+" : On emergency stop band")
                    elif valeur == 3:
                        f.write(colonne+" : On the verge")
                    elif valeur == 4:
                        f.write(colonne+" : On the sidewalk")
                    elif valeur == 5:
                        f.write(colonne+" : On bike path")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")    
                elif colonne == "catu":
                    if valeur == 1:
                        f.write(colonne+" : Driver")
                    elif valeur == 2:
                        f.write(colonne+" : Passenger")
                    elif valeur == 3:
                        f.write(colonne+" : Pedestrian")
                    elif valeur == 4:
                        f.write(colonne+" : Pedestrian in rollerblade or scooter")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")                 

                elif colonne == "grav":
                    if valeur == 1:
                        f.write(colonne+" : Unscathed")
                    elif valeur == 2:
                        f.write(colonne+" : Killed")
                    elif valeur == 3:
                        f.write(colonne+" : Hospitalized wounded")
                    elif valeur == 4:
                        f.write(colonne+" : Light injury")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "sex":
                    if valeur == 1:
                        f.write(colonne+" : Male")
                    elif valeur == 2:
                        f.write(colonne+" : Female")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "trip":
                    if valeur == 1:
                        f.write(colonne+" : Home - work")
                    elif valeur == 2:
                        f.write(colonne+" : Home - school")
                    elif valeur == 3:
                        f.write(colonne+" : Shopping - Shopping")
                    elif valeur == 4:
                        f.write(colonne+" : Professional use")
                    elif valeur == 5:
                        f.write(colonne+" : Promenade - leisure")
                    elif valeur == 9:
                        f.write(colonne+" : other")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "secu":
                    val = ""
                    if int(str(valeur)[:1]) == 1:
                        val += "Belt"
                    elif int(str(valeur)[:1]) == 2:
                        val +="Helmet"
                    elif int(str(valeur)[:1]) == 3:
                        val +="Children's device"
                    elif int(str(valeur)[:1]) == 4:
                        val +="Reflective equipment"
                    elif int(str(valeur)[:1]) == 9:
                        val += "other"
                    elif int(str(valeur)[:2]) == 1:
                        val +=", Yes"
                    elif int(str(valeur)[:2]) == 2:
                        val +=", No"
                    elif int(str(valeur)[:2]) == 3:
                        val +=", Not determinable"
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                    f.write(colonne+" : "+ val)
                elif colonne == "locp":
                    print(valeur)
                    if valeur == 1:
                        f.write(colonne+" : On pavement: A + 50 m from the pedestrian crossing")
                    elif valeur == 2:
                        f.write(colonne+" : On pavement: A - 50 m from the pedestrian crossing")
                    elif valeur == 3:
                        f.write(colonne+" : On pedestrian crossing Without light signaling")
                    elif valeur == 4:
                        f.write(colonne+" : On pedestrian crossing With light signaling")
                    elif valeur == 5:
                        f.write(colonne+" : On the sidewalk")
                    elif valeur == 6:
                        f.write(colonne+" : On the verge")
                    elif valeur == 7:
                        f.write(colonne+" : On refuge or BAU")
                    elif valeur == 8:
                        f.write(colonne+" : On against aisle")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "actp":
                    if valeur == 0:
                        f.write(colonne+" : Moving, not specified or not applicable")
                    elif valeur == 1:
                        f.write(colonne+" : Moving, Meaning bumping vehicle")
                    elif valeur == 2:
                        f.write(colonne+" : Moving, Opposite direction of the vehicle")
                    elif valeur == 3:
                        f.write(colonne+" : Various, Crossing")
                    elif valeur == 4:
                        f.write(colonne+" : Various, Masked")
                    elif valeur == 5:
                        f.write(colonne+" : Various, Playing - running")
                    elif valeur == 6:
                        f.write(colonne+" : Various, With animal")
                    elif valeur == 9:
                        f.write(colonne+" : Various, Other")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "etatp":
                    if valeur == 1:
                        f.write(colonne+" : Only")
                    elif valeur == 2:
                        f.write(colonne+" : Accompanied")
                    elif valeur == 3:
                        f.write(colonne+" : In a group")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "GP":
                    if valeur == 1:
                        f.write(colonne+" : PK or PR or increasing postal address number")
                    elif valeur == 2:
                        f.write(colonne+" : PK or PR or descending postal address number")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                elif colonne == "CATV":
                    if valeur == 1:
                        f.write(colonne+" : Bicycle")
                    elif valeur == 2:
                        f.write(colonne+" : Moped <50cm3")
                    elif valeur == 3:
                        f.write(colonne+" : Cart (Quadricycle with bodied motor) (formerly \"cart or motor tricycle\")")
                    elif valeur == 4:
                        f.write(colonne+" : Not used since 2006 (registered scooter)")
                    elif valeur == 5:
                        f.write(colonne+" : Not used since 2006 (motorcycle)")
                    elif valeur == 6:
                        f.write(colonne+" : Not used since 2006 (side-car)")
                    elif valeur == 7:
                        f.write(colonne+" : VL only")
                    elif valeur == 8:
                        f.write(colonne+" : Not used category (VL + caravan)")
                    elif valeur == 9:
                        f.write(colonne+" : Not used category (VL + trailer)")
                    elif valeur == 10:
                        f.write(colonne+" : VU only 1,5T <= GVW <= 3,5T with or without trailer (formerly VU only 1,5T <= GVW <= 3,5T)")
                    elif valeur == 11:
                        f.write(colonne+" : Most used since 2006 (VU (10) + caravan)")
                    elif valeur == 12:
                        f.write(colonne+" : Most used since 2006 (VU (10) + trailer)")
                    elif valeur == 13:
                        f.write(colonne+" : PL only 3,5T")
                    else: 
                        f.write(colonne+" : VALEUR INCONNUE")
                else:
                    f.write(colonne+" : "+str(valeur))
                f.write("\n")
            cmp +=1
        cmp = 0

In [60]:
decrypt(res)

0.65102
('locp', 0)
0
('secu', 11)
('actp', 0)
('etatp', 0)
('occutc', 0)
0.64966
('sexe', 1)
('locp', 0)
0
('actp', 0)
('etatp', 0)
('occutc', 0)
0.64564
('locp', 0)
0
('actp', 0)
('etatp', 0)
('catv', 7)
('occutc', 0)
0.65366
('obsm', 2)
('locp', 0)
0
('obs', 0)
('actp', 0)
('etatp', 0)
('occutc', 0)
0.66854
('locp', 0)
0
('actp', 0)
('agg', 1)
('etatp', 0)
('occutc', 0)
0.65098
('locp', 0)
0
('obs', 0)
('place', 1)
('actp', 0)
('etatp', 0)
('catu', 1)
('occutc', 0)
0.64448
('locp', 0)
0
('lum', 1)
('obs', 0)
('actp', 0)
('etatp', 0)
('occutc', 0)
0.63162
('locp', 0)
0
('obs', 0)
('actp', 0)
('etatp', 0)
('atm', 1)
('occutc', 0)
0.6575
('locp', 0)
0
('obs', 0)
('actp', 0)
('int', 1)
('etatp', 0)
('occutc', 0)
